In [1]:
import tensorflow as tf
import numpy as np
import sklearn.preprocessing as prep
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
def xavier_init(fan_in, fan_out, canstant = 1):
    low = -canstant*np.sqrt(6.0/(fan_in+fan_out))
    high = canstant*np.sqrt(6.0/(fan_in+fan_out))
    return tf.random_uniform((fan_in,fan_out),minval=low,maxval=high,dtype=tf.float32)

In [11]:
class AdditiveGaussianNoiseAutoencode(object):
    def __init__(self,n_input,n_hidden,transfer_function=tf.nn.softplus,optimizer=tf.train.AdamOptimizer(),scale=0.1):
        self.n_input = n_input
        self.n_hidden = n_hidden
        self.transfer = transfer_function
        self.scale = tf.placeholder(tf.float32)
        self.training_scale = scale
        network_weights = self._initialize_weights()
        self.weight = network_weights
        
        self.x = tf.placeholder(tf.float32,[None,self.n_input])
        self.hidden = self.transfer(tf.add(tf.matmul\
                                           (self.x+scale*tf.random_normal((n_input,)),self.weight['w1']),\
                                           self.weight['b1']))
        self.reconstruction = tf.add(tf.matmul(self.hidden,self.weight['w2']),self.weight['b2'])
        
        
        self.cost = 0.5*tf.reduce_sum(tf.pow(tf.subtract(self.reconstruction,self.x),2.0))
        self.optimizer = optimizer.minimize(self.cost)
        init = tf.global_variables_initializer()
        self.sess = tf.Session()
        self.sess.run(init)
    
    def _initialize_weights(self):
        all_weights = dict()
        all_weights['w1'] = tf.Variable(xavier_init(self.n_input,self.n_hidden))
        all_weights['b1'] = tf.Variable(tf.zeros([self.n_hidden],dtype=tf.float32))
        all_weights['w2'] = tf.Variable(tf.zeros([self.n_hidden,self.n_input],dtype=tf.float32))
        all_weights['b2'] = tf.Variable(tf.zeros([self.n_input],dtype=tf.float32))
        return all_weights
    
    
    def partial_fit(self,X):
        cost,opt = self.sess.run((self.cost,self.optimizer),\
                                feed_dict={self.x:X,self.scale:self.training_scale})
        return cost
    
    def calc_total_cost(self,X):
        return self.sess.run(self.cost,feed_dict={self.x:X,self.scale:self.training_scale})
    
    def transform(self,X):
        return self.sess.run(self.hidden,feed_dict={self.x:X,self.scale:self.training_scale})
    
    def generate(self,hidden = None):
        if hidden == None:
            hidden = np.random_normal(size = self.weight['b1'])
        return self.sess.run(self.reconstruction,feed_dict={self.hidden:hidden})
    
    def getweights(self):
        return self.sess.run(self.weight['w1'])
    
    def getbiases(self):
        return self.sess.run(self.weight['b1'])

In [14]:
mnist = input_data.read_data_sets('MNIST_data',one_hot=True)
def standard_scale(X_train,X_test):
    preprocessor = prep.StandardScaler().fit(X_train)
    X_train = preprocessor.transform(X_train)
    X_test = preprocessor.transform(X_test)
    return X_train,X_test

def get_random_block_from_data(data,batch_size):
    start_index = np.random.randint(0,len(data)-batch_size)
    return data[start_index:(start_index+batch_size)]

X_train,X_test = standard_scale(mnist.train.images,mnist.test.images)

n_samples = int(mnist.train.num_examples)
training_epochs = 20
batch_size = 128
display_step = 1

autoencoder = AdditiveGaussianNoiseAutoencode(n_input=784,n_hidden=200,\
                                             transfer_function = tf.nn.softplus,\
                                            optimizer = tf.train.AdamOptimizer(learning_rate=0.001),\
                                            scale = 0.01)


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [15]:
for epoch in range(training_epochs):
    avg_cost = 0.
    total_batch = int(n_samples/batch_size)
    for i in range(total_batch):
        batch_xs = get_random_block_from_data(X_train,batch_size)
        
        cost = autoencoder.partial_fit(batch_xs)
        avg_cost += cost / n_samples * batch_size
        
        
    if epoch % display_step == 0:
        print("Epoch:",'%04d' % (epoch+1),\
             "Cost:","{:.9f}".format(avg_cost))
        
        
print ("Total cost: " + str(autoencoder.calc_total_cost(X_test)))

('Epoch:', '0001', 'Cost:', '19695.067627273')
('Epoch:', '0002', 'Cost:', '12399.552494318')
('Epoch:', '0003', 'Cost:', '10705.669564205')
('Epoch:', '0004', 'Cost:', '9686.600039773')
('Epoch:', '0005', 'Cost:', '9891.139448295')
('Epoch:', '0006', 'Cost:', '9761.786105114')
('Epoch:', '0007', 'Cost:', '9414.254188636')
('Epoch:', '0008', 'Cost:', '9447.903031818')
('Epoch:', '0009', 'Cost:', '8339.237385227')
('Epoch:', '0010', 'Cost:', '8123.087101136')
('Epoch:', '0011', 'Cost:', '9028.699749432')
('Epoch:', '0012', 'Cost:', '7989.269300000')
('Epoch:', '0013', 'Cost:', '8066.098240341')
('Epoch:', '0014', 'Cost:', '7925.840646591')
('Epoch:', '0015', 'Cost:', '7618.230105682')
('Epoch:', '0016', 'Cost:', '8229.501899432')
('Epoch:', '0017', 'Cost:', '8534.381428977')
('Epoch:', '0018', 'Cost:', '7709.351215341')
('Epoch:', '0019', 'Cost:', '7740.514997727')
('Epoch:', '0020', 'Cost:', '8090.264693750')
Total cost: 679826.0
